Простите, пожалуйста, что сдаю так сильно позже дедлайна и так плохо написанную работу :(
Я постараюсь в следующий раз сделать всё лучше или хотя бы в срок.

In [1]:
from math import log

In [2]:
from collections import Counter

In [3]:
import pandas as pd

In [4]:
import re

In [5]:
import numpy as np

In [6]:
import pymorphy2

In [ ]:
def tf(term, doc):
    c = Counter()
    for word in doc:
        c[word] += 1
    return c[term] / len(doc)

In [ ]:
def idf(term, corp):
    n = 0
    for doc in corp.values():
        if term in doc:
            n += 1
    return log((len(corp) - n + 0.5) / (n + 0.5))

In [ ]:
def bm25(terms, doc, corp, k=2.0, b=0.75):
    avgld = sum([len(doc) for doc in corp.values()])/len(corp)
    return sum([idf(term,corp) * (tf(term, doc) * (k + 1)/ (tf(term, doc) \
                                + k * (1 - b + b * (len(doc)/avgld)))) for term in terms])

In [ ]:
data = pd.read_csv('quora_question_pairs_rus.csv',encoding='utf-8')

In [ ]:
data.head()

,Unnamed: 0,question1,question2,is_duplicate
0,0,Какова история кохинор кох-и-ноор-бриллиант,"что произойдет, если правительство Индии украд...",0
1,1,как я могу увеличить скорость моего интернет-с...,как повысить скорость интернета путем взлома ч...,0
2,2,"почему я мысленно очень одинок, как я могу это...","найти остаток, когда математика 23 ^ 24 матема...",0
3,3,которые растворяют в воде быстро сахарную соль...,какая рыба выживет в соленой воде,0
4,4,астрология: я - луна-колпачок из козерога и кр...,Я тройная луна-козерог и восхождение в козерог...,1


In [ ]:
queries = list(data['question1'])

In [ ]:
corpus = list(data['question2'])

In [ ]:
morph = pymorphy2.MorphAnalyzer()
def normalize(doc):
    tokens = re.sub('  ', ' ', re.sub(r'[^\w\s]','',doc)).split()
    lemmas = []
    for i, token in enumerate(tokens):
        tokens[i] = token.lower()
        lemmas.append(morph.parse(token)[0].normal_form)
    return lemmas

In [ ]:
def preprocessing(corpus):
    raw_texts = {}
    morph = pymorphy2.MorphAnalyzer()
    num = 0
    for doc in corpus:
        if type(doc) != float:
            raw_texts[num] = normalize(doc)
        num += 1
    return raw_texts

In [ ]:
preprocessing(corpus[:100])

{0: ['что',
  'произойти',
  'если',
  'правительство',
  'индия',
  'украсть',
  'кохинор',
  'кохинооралмаз',
  'назад'],
 1: ['как',
  'повысить',
  'скорость',
  'интернет',
  'путём',
  'взлом',
  'через',
  'dns'],
 2: ['найти',
  'остаток',
  'когда',
  'математика',
  '23',
  '24',
  'математика',
  'разделить',
  'на',
  '24',
  '23'],
 3: ['какой', 'рыба', 'выжить', 'в', 'солёный', 'вода'],
 4: ['я',
  'тройной',
  'лунакозерог',
  'и',
  'восхождение',
  'в',
  'козерог',
  'что',
  'это',
  'говорить',
  'о',
  'я'],
 5: ['что',
  'делать',
  'ребёнок',
  'активный',
  'и',
  'далёкий',
  'от',
  'телефонный',
  'и',
  'видеоигра'],
 6: ['что', 'я', 'должный', 'делать', 'чтобы', 'быть', 'великое', 'геолог'],
 7: ['когда', 'вы', 'использовать', 'вместо'],
 8: ['как',
  'я',
  'мочь',
  'взломать',
  'motorola',
  'dcx3400',
  'для',
  'бесплатный',
  'интернет'],
 9: ['что',
  'некоторый',
  'технический',
  'специалист',
  'мочь',
  'рассказать',
  'о',
  'долговечность',
 

In [ ]:
def search(query, corp):
    bms = []
    for num in corp:
        bms.append([bm25(query, corp[num], corp), num])
    result = sorted(bms)
    return result

In [ ]:
def check(queries, corpus, data):
    outdict = {}
    corp = preprocessing(corpus[:10000])
    queries = queries[:1000] #Делать размер корпуса и листа запросов больше я побоялась, 
                            #потому что даже это отрабатывало целую вечность.
                            #И решила, что пусть лучше на неполных данных, но отработает.
    for query in queries:
        outdict[query] = search(normalize(query), corp)[-5:]
    exacts = 0
    for query in outdict:
        i = 0
        for res in list(outdict[query]):
            needed = data[data['Unnamed: 0'] == res[1]]
            if query == needed['question1'][res[1]] and needed['is_duplicate'][res[1]] == 1 and i < 1:
                i = 1
        exacts = exacts + i
    return exacts / len(queries)

In [ ]:
check(queries, corpus, data)

Задание 2

In [ ]:
for res in search(normalize('рождественские каникулы'), preprocessing(corpus[:10000]))[-10:]: #То же самое про размер корпуса.
    num = res[1]
    print(data[data['Unnamed: 0'] == num]['question2'][num], '\nBM25:', res[0])

Задание 3

In [ ]:
def search_also(query, corp, bm):
    bms = []
    for num in corp:
        bms.append([bm(query, corp[num], corp), num])
    result = sorted(bms)
    return result

In [ ]:
def bm15(query, doc, corp):
    return bm25(query, doc, corp, b=0)

In [ ]:
def bm11(query, doc, corp):
    return bm25(query, doc, corp, b=1)

In [ ]:
def check(queries, corpus, data, bm):
    outdict = {}
    corp = preprocessing(corpus[:1000])
    queries = queries[:100]
    for query in queries:
        outdict[query] = search_also(normalize(query), corp, bm)[-5:]
    exacts = 0
    for query in outdict:
        i = 0
        for res in list(outdict[query]):
            needed = data[data['Unnamed: 0'] == res[1]]
            if query == needed['question1'][res[1]] and needed['is_duplicate'][res[1]] == 1 and i < 1:
                i = 1
        exacts = exacts + i
    return exacts / len(queries)

In [ ]:
check(queries, corpus, data, bm11)

In [ ]:
check(queries, corpus, data, bm15)